In [20]:
import glob 
import zipfile
import numpy as np
import cv2
import os

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd

zip_files = glob.glob('drive/My Drive/colab_data/dogs-vs-cats/*.zip')

if(not os.path.exists("drive/My Drive/colab_data/dogs-vs-cats/data/train")):
  for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zf:
      zf.extractall("drive/My Drive/colab_data/dogs-vs-cats/data")

print("extraction complete")

train_path = 'drive/My Drive/colab_data/dogs-vs-cats/data/train'
test_path = 'drive/My Drive/colab_data/dogs-vs-cats/data/test1'
IMAGE_SIZE = (50, 50)

train_data = []
def make_training_data():
    LABELS = {'cat': 0, 'dog': 1}
    for f in tqdm(os.listdir(train_path)):
        path = os.path.join(train_path, f)
        if("jpg" not in f):
            continue
        img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img, IMAGE_SIZE)
        train_data.append([img, np.eye(len(LABELS))[LABELS[f.split('.')[0]]]])
    np.save("drive/My Drive/colab_data/dogs-vs-cats/training_data.npy", train_data)

if(os.path.exists("drive/My Drive/colab_data/dogs-vs-cats/training_data.npy")):
    train_data = np.load("drive/My Drive/colab_data/dogs-vs-cats/training_data.npy", allow_pickle=True)
else:
    make_training_data()

np.random.shuffle(train_data)
print(train_data[0])

class Net(nn.Module):
    def __init__(self):
        super().__init__() 
        self.conv1 = nn.Conv2d(1, 32, 5) 
        self.conv2 = nn.Conv2d(32, 64, 5) 
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        self.fc1 = torch.nn.Linear(2*2*128, 32)
        self.fc2 = torch.nn.Linear(32, 2) 
        
        self.max_pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        x = F.relu(self.fc1(x.view(-1, 128*2*2)))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

net = Net().to(device)

train_X = torch.Tensor([i[0] for i in train_data]).view(-1,50,50)
train_X = train_X/255.0
train_y = torch.Tensor([i[1] for i in train_data])

def train(net):
    BATCH_SIZE = 64
    EPOCHS = 20
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss_function = nn.MSELoss()
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)): 
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            net.zero_grad()

            optimizer.zero_grad()   # zero the gradient buffers
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()

            del(batch_X)
            del(batch_y)

        print(f"Epoch: {epoch}. Loss: {loss}")
        
train(net)

predictions=[]
id_line=[]
def test(net):
    for f in os.listdir(test_path):
        id_line.append(f.split('.')[0])
        path = os.path.join(test_path, f)
        img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img, (50, 50))
        test_X = torch.Tensor(img).view(-1, 1, 50, 50)
        test_X = test_X.to(device)
        net_out = net(test_X)
        
        predictions.append(torch.argmax(net_out))
        del(test_X)

test(net)

predicted_val = [x.tolist() for x in predictions]
submission_df = pd.DataFrame({'id':id_line, 'label':predicted_val})
submission_df.to_csv("submiss.csv", index=False)

extraction complete
[array([[ 94,  41,  45, ..., 134, 150, 150],
       [ 93,  59,  69, ..., 138, 148, 150],
       [ 92,  58,  64, ..., 138, 146, 146],
       ...,
       [ 69,  58,  93, ..., 100,  96,  55],
       [125,  88, 106, ..., 156, 144,  99],
       [ 90,  99,  79, ..., 138,  92, 103]], dtype=uint8)
 array([0., 1.])]
Running on the GPU
11996954624 11662262272 11662165504 104960


RuntimeError: ignored